In [334]:
from __future__ import division
from pyomo.environ import *
from pyomo.core.base.expr import identify_variables
import numpy as np

model = ConcreteModel()


model.n = range(4)

model.x = Var(model.n, within=Binary)

model.c = np.array([1,2,2,1])
# model.a1 = np.matrix([[-1,-1,0,0]])
# model.a2 = np.matrix([[0,0,-1,-1]])
model.a = np.matrix([[-1,-1,0,0],[0,0,-1,-1]])
# model.b1 = np.array([-1])
# model.b2 = np.array([-2])
model.b = np.array([-1,-2])
# model.set1 = range(len(model.b1))
# model.set2 = range(len(model.b2))
model.eqmat = np.matrix([1,1,1,1])
model.eqb = np.array([3])
model.set = range(len(model.b))
consets = [[0],[1]]
varsets = [[0,1],[2,3]]

In [335]:
def projectBinary(x):
    for row in range(len(x)):
        if x[row] <= 0.5:
            x[row] = 0
        else:
            x[row] = 1
    return x

def projectHyperplane(x,A,b):
    for row in range(A.shape[0]):
        if np.dot(A[row,:],x) - b[row] > 0:
            x = x - ((( (np.dot(np.array(A[row,:]),x))[0] - b[row] )/ ((np.array(np.square(A).sum(axis=1))[row])[0]) ) * np.array(A[row,:].flatten())[0])
    return x

def find_points(A,b):
    """Finds feasible points for a set of linear constraints"""
    points = []
    for i in range(100):
        x = np.random.rand(A.shape[1])
        x_old = x+1
        while (np.linalg.norm(x-x_old) > 0.00001):
            x_old = x
            x = projectHyperplane(x,A,b)
            x = projectBinary(x)
        points.append(x)
        
    return np.unique(np.array(points),axis=0)

points = []
for i in range(len(varsets)):
    points.append(find_points(model.a[consets[i],varsets[i]],model.b[consets[i]]))
print(points)

[array([[0., 1.],
       [1., 0.],
       [1., 1.]]), array([[1., 1.]])]


In [329]:
points
points1 = [np.array([[1,1],[0,1]]),np.array([[1,1]])]
print(points1)
points = points1

[array([[1, 1],
       [0, 1]]), array([[1, 1]])]


In [336]:
def create_A_and_rhs(eqmat,eqb,points,varsets):
    big_matrix = np.zeros((len(eqb)+len(points),1))
    for count,set_points in enumerate(points):
        matrix_of_set_i = np.zeros((len(eqb)+len(points),len(set_points)))
        # Now populate the matrix
        matrix_of_set_i[count+1,:] = 1
        for j in range(len(eqb)):
            for column,point in enumerate(set_points):
                matrix_of_set_i[j,column] = np.dot(eqmat[j,varsets[count]],point)[0,0]
        big_matrix = np.concatenate((big_matrix,matrix_of_set_i),axis=1)
    big_matrix = np.delete(big_matrix,0,axis=1)

    rhs = np.concatenate((model.eqb,np.ones(len(points))))
    return big_matrix,rhs

big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
print(big,rhs)

(array([[1., 1., 2., 2.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.]]), array([3., 1., 1.]))


In [337]:
def get_obj_values(c,points,varsets):
    obj_array = np.zeros(1)
    for count,set_points in enumerate(points):
        for point in set_points:
            obj = np.dot(c[varsets[count]],point)
            obj_array = np.append(obj_array,obj)
    obj_array = np.delete(obj_array,0)
    return obj_array

print(get_obj_value(model.c,points,varsets))

[2. 1. 3. 3.]


In [338]:
range(big.shape[1])

[0, 1, 2, 3]

In [339]:
rmlp = ConcreteModel()
rmlp.n = range(big.shape[1])
rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
rmlp.con = ConstraintList()
big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
obj = get_obj_values(model.c,points,varsets)
for i in range(big.shape[0]):
    rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Initializing dual steep norms . . .


In [243]:
for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

('Variable component object', <pyomo.core.base.var.IndexedVar object at 0x000000000A7DBB38>)
('   ', 0, 0.0)
('   ', 1, 1.0)
('   ', 2, 1.0)


In [208]:
print ("Duals")
from pyomo.core import Constraint
pi = np.zeros(len(model.eqb))
mu = np.zeros(len(varsets))
print(mu)
for c in rmlp.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(rmlp, str(c))
    print(cobject)
    for index in cobject:
        print(cobject[index])
        if index <= len(model.eqb):
            pi[index-1] = rmlp.dual[cobject[index]]
        else:
            mu[index-1-len(pi)] = rmlp.dual[cobject[index]]
        print ("      ", index, rmlp.dual[cobject[index]])

Duals
[0. 0.]
('   Constraint', <pyomo.core.base.constraint.ConstraintList object at 0x000000000A8F8D30>)
con
con[1]
('      ', 1, 0.0)
con[2]
('      ', 2, 1.0)
con[3]
('      ', 3, 3.0)


In [202]:
print(pi)
print(mu)
print(varsets)

[0.]
[2. 3.]
[[0, 1], [2, 3]]


In [332]:
def get_c_A_values(point,setnum,varsets,eqmat,eqb,c):
    obj = np.dot(c[varsets[setnum]],point)
    Acol = np.zeros((len(eqb)+len(varsets),1))
    Acol[setnum+len(eqb),0] = 1
    for i in range(len(eqb)):
        Acol[i,0] = np.dot(eqmat[i,varsets[setnum]],point)[0,0]
    return obj,Acol

In [209]:
# Now create the subproblem and use the dual values to solve
for k,varset in enumerate(varsets):
    subprob = ConcreteModel()
    subprob.x = Var(range(len(varset)),within=Binary)
    subprob.con = ConstraintList()
    for set_num,i in enumerate(consets[k]):
        subprob.con.add(expr = 
                       sum(model.a[i,j]*subprob.x[col] for col,j in enumerate(varset)) <= model.b[i]
                       )
#     print(k)
#     print(model.c[varsets[k]])
#     print(model.c[varsets[k]] - np.dot(pi,model.a[consets[k],varsets[k]])[0,0])
    constants = model.c[varsets[k]] - np.dot(pi,model.a[consets[k],varsets[k]])[0,0]
#     print(constants[0])
    subprob.OBJ = Objective(expr = 
                           sum(constants[i]*subprob.x[i] for i in range(len(varset))) - mu[k]
                           )
    solver = SolverFactory('cplex')
    results = solver.solve(subprob, tee = True)
    subprob.solutions.load_from(results)
    if subprob.OBJ() < 0:
        print(subprob.OBJ())
        xval = []
    #     print(subprob.component_objects(Objective,active=True))
        for v in subprob.component_objects(Var, active=True):
            print("Variable component object",v)
            for index in v:
                xval.append(v[index].value)
                print("   ", index, v[index].value)
        newc,newAcol = get_c_A_values(np.array(xval),k,varsets,model.eqmat,model.eqb,model.c)
        obj = np.append(obj,newc)
        big = np.append(big,newAcol,axis=1)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpdvcotk.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpqlsk1r.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpqlsk1r.pyomo.lp
Objective sense      : Minimize
Variables            :       3  [Nneg: 1,  Binary: 2]
Objective nonzeros   :       3
Linear constraints   :       2  [Less: 1,  Equal: 1]
  Nonzeros           :       3
  RHS nonzeros       :       2

Variables            : Min LB: 0.0000000        Max UB: 1.000000       
Objective nonzeros   : Min   : 

In [205]:
range(big.shape[1])

[0, 1, 2, 3]

In [206]:
rmlp = ConcreteModel()
rmlp.n = range(big.shape[1])
rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
rmlp.con = ConstraintList()
# big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
# obj = get_obj_values(model.c,points,varsets)
for i in range(big.shape[0]):
    rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Initializing dual steep norms . . .


In [207]:
for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

('Variable component object', <pyomo.core.base.var.IndexedVar object at 0x000000000A8F8748>)
('   ', 0, 0.0)
('   ', 1, 0.0)
('   ', 2, 1.0)
('   ', 3, 1.0)


In [333]:
rmlp = ConcreteModel()
rmlp.n = range(big.shape[1])
rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
rmlp.con = ConstraintList()
big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
obj = get_obj_values(model.c,points,varsets)
for i in range(big.shape[0]):
    rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = False)
rmlp.solutions.load_from(results)

convergenceflag = 0
while convergenceflag == 0:
    convergenceflag = 1
    print ("Duals")
    from pyomo.core import Constraint
    pi = np.zeros(len(model.eqb))
    mu = np.zeros(len(varsets))
    for c in rmlp.component_objects(Constraint, active=True):
        cobject = getattr(rmlp, str(c))
        for index in cobject:
            if index <= len(model.eqb):
                pi[index-1] = rmlp.dual[cobject[index]]
            else:
                mu[index-1-len(pi)] = rmlp.dual[cobject[index]]
            print ("      ", index, rmlp.dual[cobject[index]])
            
    # Now create the subproblem and use the dual values to solve
    for k,varset in enumerate(varsets):
        subprob = ConcreteModel()
        subprob.x = Var(range(len(varset)),within=Binary)
        subprob.con = ConstraintList()
        for set_num,i in enumerate(consets[k]):
            subprob.con.add(expr = 
                           sum(model.a[i,j]*subprob.x[col] for col,j in enumerate(varset)) <= model.b[i]
                           )
        constants = model.c[varsets[k]] - np.dot(pi,model.a[consets[k],varsets[k]])[0,0]
        subprob.OBJ = Objective(expr = 
                               sum(constants[i]*subprob.x[i] for i in range(len(varset))) - mu[k]
                               )
        solver = SolverFactory('cplex')
        results = solver.solve(subprob, tee = True)
        subprob.solutions.load_from(results)
        if subprob.OBJ() < 0:
            print("Does this print??")
            convergenceflag = 0
            print(subprob.OBJ())
            xval = []
            for v in subprob.component_objects(Var, active=True):
                for index in v:
                    xval.append(v[index].value)
            newc,newAcol = get_c_A_values(np.array(xval),k,varsets,model.eqmat,model.eqb,model.c)
            obj = np.append(obj,newc)
            big = np.append(big,newAcol,axis=1)
            print(points[k])
            print(np.array(xval))
            points[k] = np.append(points[k],[np.array(xval)],axis=0)
    
    rmlp = ConcreteModel()
    rmlp.n = range(big.shape[1])
    rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
    rmlp.con = ConstraintList()
    for i in range(big.shape[0]):
        rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

    rmlp.OBJ = Objective(expr = 
                        sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

    rmlp.dual = Suffix(direction=Suffix.IMPORT)
    solver = SolverFactory('cplex',solver_io='nl')
    solver.options['presolve'] = 0
    solver.options['lpdisplay'] = 2
    results = solver.solve(rmlp, tee = False)
    rmlp.solutions.load_from(results)
    
for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

Duals
('      ', 1, 0.0)
('      ', 2, 2.0)
('      ', 3, 3.0)

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpntgja3.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpgvhi0j.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpgvhi0j.pyomo.lp
Objective sense      : Minimize
Variables            :       3  [Nneg: 1,  Binary: 2]
Objective nonzeros   :       3
Linear constraints   :       2  [Less: 1,  Equal: 1]
  Nonzeros           :       3
  RHS nonzeros       :       2

Variables            : Min LB: 0.0000000

CPLEX> ('Variable component object', <pyomo.core.base.var.IndexedVar object at 0x000000000AD025C0>)
('   ', 0, 0.0)
('   ', 1, 0.0)
('   ', 2, 1.0)
('   ', 3, 1.0)
